##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 初心者のための TensorFlow 2.0 入門

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">View on TensorFlow.org</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a>   </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">View source on GitHub</a>   </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Download notebook</a>   </td>
</table>

このガイドでは簡単な導入として、[Keras](https://www.tensorflow.org/guide/keras/overview) を使って次のことを行います。

1. 画像を分類するニューラルネットワークを構築する。
2. そのニューラルネットワークをトレーニングする。
3. 最後に、モデルの精度を評価する。
4. モデルの精度を評価します。

このファイルは [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) の notebook ファイルです。 Python プログラムはブラウザ上で直接実行されます。TensorFlow を学んだり使ったりするには最良の方法です。Google Colab のnotebook の実行方法は以下のとおりです。

1. Pythonランタイムへの接続：メニューバーの右上で「接続」を選択します。
2. ノートブックのコードセルをすべて実行：「ランタイム」メニューから「すべてのセルを実行」を選択します。

## TensorFlow をセットアップする

まず、TensorFlow をプログラムにインポートします。

In [ ]:
import tensorflow as tf

[Colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb) ではなく独自の開発環境で手順を実行している場合は、開発用 TensorFlow のセットアップについて、[インストールガイド](https://www.tensorflow.org/install)をご覧ください。

注意: 独自の開発環境を使用している場合は、TensorFlow 2 パッケージをインストールする際に、最新の `pip` にアップグレード済みであることを確認してください。詳細は、[インストールガイド](https://www.tensorflow.org/install)をご覧ください。

## データセットを読み込む

[MNIST データセット](http://yann.lecun.com/exdb/mnist/)を読み込んで準備します。サンプルデータを整数から浮動小数点数に変換します。

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## 機械学習モデルを構築する

層を積み重ねて`tf.keras.Sequential`モデルを構築します。訓練のためにオプティマイザと損失関数を選びます。

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

各サンプルについて、モデルは "[logits](https://developers.google.com/machine-learning/glossary#logits)" または "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" スコアのベクトルをクラスごとに返します。

In [ ]:
predictions = model(x_train[:1]).numpy()
predictions

`tf.nn.softmax` 関数は、クラスごとにこれらのロジットを "probabilities" に変換します。 

In [ ]:
tf.nn.softmax(predictions).numpy()

注意: この `tf.nn.softmax` をネットワークの最後のレイヤーのアクティベーション関数として組み込むことも可能です。こうすることでモデルの出力をより直接的に解釈可能にすることもできますが、softmax 出力を使用する場合、すべてのモデルに対して正確で数値的に安定した損失計算を提供することは不可能であるため、この方法は推奨されません。 

`losses.SparseCategoricalCrossentropy` 損失は、ロジットのベクトルと `True` インデックスを取り、各サンプルのスカラー損失を返します。

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

この損失は、True のクラスの負の対数確率に等しくなります。モデルが正しいクラスであることが確実な場合はゼロです。

トレーニングされていないこのモデルでは、ランダムに近い確率（クラス当たり 1/10）が得られるため、最初の損失は `-tf.math.log(1/10) ~= 2.3` に近くなります。

In [ ]:
loss_fn(y_train[:1], predictions).numpy()

トレーニングを開始する前に、Keras `Model.compile` を使用してモデルの構成とコンパイルを行います。[`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) クラスを `adam` に、`loss` を前に定義した `loss_fn` 関数に設定し、`metrics` パラメータを `accuracy` に設定して評価するモデルの指標を指定します。

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## モデルをトレーニングして評価する

損失を最小限に抑えられるようにモデルのパラメータを `Model.fit` メソッドで調整します。 

In [ ]:
model.fit(x_train, y_train, epochs=5)

`Model.evaluate` メソッドは通常、[検証セット](https://developers.google.com/machine-learning/glossary#validation-set)が[テストセット](https://developers.google.com/machine-learning/glossary#test-set)でモデルのパフォーマンスをチェックします。

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

`tf.nn.softmax` 関数はクラスごとにこれらのロジットを "確率" に変換します。

モデルが確率を返すようにするには、トレーニング済みのモデルをラップして、それに softmax を接続することができます。

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
probability_model(x_test[:5])

## まとめ

おめでとうございます！事前構築済みのデータセットと [Keras](https://www.tensorflow.org/guide/keras/overview) API を使用して、機械学習モデルをトレーニングしました。

Keras のその他の使用例については、[チュートリアル](https://www.tensorflow.org/tutorials/keras/)をご覧ください。Keras を使ったモデルの構築に関してさらに学習するには、[ガイド](https://www.tensorflow.org/guide/keras)をお読みください。データの読み込みと準備についてさらに学習するには、[画像データの読み込み](https://www.tensorflow.org/tutorials/load_data/images)または [CSV データの読み込み](https://www.tensorflow.org/tutorials/load_data/csv)に関するチュートリアルをご覧ください。
